In [1]:
# imports
from tqdm import tqdm #progress bar
import math
import statistics
import pandas as pd
import numpy as np
from uncertainties import ufloat
from uncertainties.umath import * 

# SWEET-Cat
import urllib
from astropy import coordinates as coord
from astropy import units as u

from astropy.table import Table
from astropy.io.ascii import convert_numpy

#Download SWEET-Cat
sweetCat_table_url = "https://sweetcat.iastro.pt/catalog/SWEETCAT_Dataframe.csv"
converters={'gaia_dr2': [convert_numpy(np.int64)],'gaia_dr3': [convert_numpy(np.int64)] }
sweet_table = Table.read(sweetCat_table_url, encoding='UTF-8',format='csv', converters=converters)
sweet_stars = sweet_table['Name']

#Display
from IPython.display import display, Math, Latex

# astroquery
from astroquery.simbad import Simbad
from astroquery.utils.tap.core import TapPlus #tap service
simbadtap = TapPlus(url="http://simbad.u-strasbg.fr/simbad/sim-tap")

In [2]:
df = pd.read_csv('_final_.txt', sep='\t')

In [3]:
df

,Star,average: vsini (km/s),average: err_1,average: err_2,standard deviation: vsini (km/s),standard deviation: err_1,standard deviation: err_2,median: vsini (km/s),median: err_1,median: err_2,nasa: vsini (km/s),nasa: err_1,nasa: err_2,nasa: reference
0,11 Com,5.292000,NaN,NaN,1.615000,NaN,NaN,1.200,NaN,NaN,1.2,1.0,-1.0,<a refstr=LIU_ET_AL__2008 href=https://ui.adsa...
1,11 UMi,4.825000,NaN,NaN,1.433333,NaN,NaN,1.500,NaN,NaN,NaN,NaN,NaN,<a refstr=STASSUN_ET_AL__2017 href=https://ui....
2,14 And,4.488333,NaN,NaN,1.386000,NaN,NaN,1.515,NaN,NaN,2.6,NaN,NaN,<a refstr=SATO_ET_AL__2008 href=https://ui.ads...
3,14 Her,1.966667,NaN,NaN,1.940000,NaN,NaN,2.000,NaN,NaN,NaN,NaN,NaN,<a refstr=FENG_ET_AL__2022 href=https://ui.ads...
4,16 Cyg B,9.765556,NaN,NaN,3.281176,NaN,NaN,2.200,NaN,NaN,NaN,NaN,NaN,<a refstr=STASSUN_ET_AL__2017 href=https://ui....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3798,K2-405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<a refstr=CHRISTIANSEN_ET_AL__2022 href=https:...
3799,K2-406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<a refstr=CHRISTIANSEN_ET_AL__2022 href=https:...
3800,K2-407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<a refstr=CHRISTIANSEN_ET_AL__2022 href=https:...
3801,K2-408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<a refstr=CHRISTIANSEN_ET_AL__2022 href=https:...


In [4]:
print(sweet_table.columns)

<TableColumns names=('Name','hd','RA','DEC','Vmag','eVmag','PlxFlag','Teff','eTeff','Logg','eLogg','Vt','eVt','[Fe/H]','e[Fe/H]','Reference','Link','SWFlag','Update','Comment','Database','gaia_dr2','gaia_dr3','Plx','ePlx','Gmag','eGmag','RPmag','eRPmag','BPmag','eBPmag','FG','eFG','G_flux_std_n','Logg_gaia','eLogg_gaia','Mass_t','eMass_t','Radius_t','eRadius_t','spec_base','Distance','RA_EU','DEC_EU','RA_NASA','DEC_NASA','Distance_b','eDistance_b')>


In [71]:
def star_prop(star, df):
    
    if star == 'HD 286123':
        star = 'K2-232'
    if star == '38Vir':
        star = 'HD 111998'
    if star == 'TOI-130':
        star = 'HD 5278'
    
    
    #print(sweet_table[sweet_table['Name'] == star]['Teff'])
    Teff = sweet_table[sweet_table['Name'] == star]['Teff'][0]
    eTeff = sweet_table[sweet_table['Name'] == star]['eTeff'][0]
    Logg = sweet_table[sweet_table['Name'] == star]['Logg'][0]
    eLogg = sweet_table[sweet_table['Name'] == star]['eLogg'][0]
    Vt = sweet_table[sweet_table['Name'] == star]['Vt'][0]
    eVt = sweet_table[sweet_table['Name'] == star]['eVt'][0]
    Fe_H = sweet_table[sweet_table['Name'] == star]['[Fe/H]'][0]
    eFe_H =  sweet_table[sweet_table['Name'] == star]['e[Fe/H]'][0]
    
    vsini = float(df[df['Star'] == star]['nasa: vsini (km/s)'])
    #print(float(vsini))
    e1vsini = float(df[df['Star'] == star]['nasa: err_1'])
    e2vsini = float(df[df['Star'] == star]['nasa: err_2'])
    
    display(Math("T_{eff} =" + str(Teff) + "\pm" + str(eTeff) + " K"))
    display(Math("log(g) =" + str(Logg) + "\pm" + str(eLogg)))
    display(Math("\\xi =" + str(Vt) + "\pm" + str(eVt) + " km/s"))
    display(Math("[Fe/H] =" + str(Fe_H) + "\pm" + str(eFe_H) + " km/s"))
    display(Math("v sini =" + str(vsini) + "\pm^{" + str(e1vsini) + "}_{" + str(e2vsini) + "} km/s"))

    return Teff, eTeff, Logg, eLogg, Vt, eVt, Fe_H, eFe_H, vsini, e1vsini, e2vsini

In [72]:
#Teff, eTeff, Logg, eLogg, Vt, eVt, Fe_H, eFe_H, vsini, e1vsini, e2vsini = star_prop('HD60532', df)

In [73]:
spectra_stars = ['HD 19994 A', 'HD 60532', 'HR 858', 'HD 10647', 'HD 208487', '38 Vir', 'HD 103774 A', 'HD 50499', 'HD 205739', 'WASP-166', 'HD 11231', 'HD 147873', 'KELT-6', 'K2-100', 'GJ 3021', 'WASP-20 A', 'K2-290', 'WASP-84', 'WASP-174', 'WASP-31', 'WASP-190', 'HD 286123', 'TOI-130', 'HD 70573', 'HD 196885 A', 'DS Tuc A', 'HD 13908', 'HATS-3', 'HD 66141', 'TOI-677', 'HD 143105', 'WASP-136', 'HD 63433', 'Kepler-65', 'CoRoT-25', 'HD 87646 A', 'KELT-4 A', 'HD 12484', 'WASP-103', 'WASP-138', 'WASP-120', 'HD 133131 B', 'HD 106315', 'Kepler-21', 'HD 115954', 'HD 27969', 'WASP-118', 'HAT-P-50', 'HATS-10', 'HD 93963 A', 'HATS-68', 'HAT-P-34', 'WASP-114', 'XO-7', 'KELT-16', 'HAT-P-45', 'CoRoT-6', 'WASP-75', 'WASP-87 A', 'CoRoT-4', 'WASP-90', 'WASP-153']

In [74]:
count_T = 0
count_F = 0
not_in_sc = []
for i in spectra_stars:
    if i == 'HD 286123':
        i = 'K2-232'
    if i == '38Vir':
        i = 'HD 111998'
    if i == 'TOI-130':
        i = 'HD 5278'

    if i in sweet_table['Name']:
        count_T += 1
    if i not in sweet_table['Name']:
        not_in_sc.append(i)
        count_F += 1
print(count_T)
print(count_F)
print(not_in_sc)

62
0
[]


In [75]:
# nasa = pd.read_csv('nasa_ps.csv')
# count = 0
# sweet_gaia = []
# for star_name in not_in_sc:        
#     print(star_name in list(nasa['hostname']))
#     if star_name in list(nasa['hostname']):
#         gaia = list(nasa[nasa['hostname']==star_name]['gaia_id'])
#         id2 = gaia[0][9:]
#         sweet_gaia.append(id2)
#         print(float(id2) in sweet_table['gaia_dr2'])
#         new_name = list(sweet_table[sweet_table['gaia_dr2']==float(id2)]['Name'])[0]
#         print(star_name, new_name)
#     if star_name not in list(nasa['hostname']):
#         count += 1
#         print(star_name)
# print(count)
# print(sweet_gaia)

In [76]:
rows = 
for star_name in spectra_stars:
    display('Star: ' + star_name)
    Teff, eTeff, Logg, eLogg, Vt, eVt, Fe_H, eFe_H, vsini, e1vsini, e2vsini = star_prop(star_name, df)    

'Star: HD 19994 A'

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>